In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('data/final_dataframe.csv', index_col=0, parse_dates=['date'])
df.tail()

In [ ]:
df.loc[df.country=='United_States_of_America'].cases[::-1].cumsum()[::-1]

In [ ]:
data = df.groupby(['country','date'])['cases'].sum().groupby(level=[0]).cumsum()

In [ ]:
pd.DataFrame(data)